## ORGANS: Transplant count comparison

---

### Initialize environment

In [2]:
library(tidyverse)
library(lubridate)

In [3]:
root_path     <- here::here()
data_path     <- paste0(root_path, "/01_data")
analysis_path <- paste0(root_path, "/02_analysis")

In [4]:
# import necessary data
setwd(data_path)

optn <- readRDS("clean/optn_tx_counts_clean.rds")

transplant <- readRDS("clean/transplant_clean.rds")

state_code_mapping <- read.csv("raw/00_misc/state_code_mapping_20221022.csv")

setwd(analysis_path)

### Calculate number of transplants for each state during the different periods

1. `1YR19vs1YR21` = full year of 2019 vs. full year of 2021
2. `2YR1819vs2YR2122` = combined full years of 2018-2019 vs. full years of 2021-2022
3. `730Dvs730D` = UNOS definitions of 730-day periods pre- and post-policy

Note the UNOS 730-day definitions are actually 731-day periods, but we'll keep to be consistent.

In [5]:
# count transplants by period/state for '19 vs. '21 (using srtr transplant data)
txByStatePeriod.v1 <- transplant %>%
    filter(REC_TX_DT_YEAR %in% c(2019, 2021),
           !is.na(CAN_HOSPITAL_STATE)) %>%
    mutate(POLICY_ERA = if_else(REC_TX_DT_YEAR == 2019, "PRE", "POST"),
           START_DATE = as.Date(paste0(REC_TX_DT_YEAR, "-01-01")),
           END_DATE = as.Date(paste0(REC_TX_DT_YEAR, "-12-31"))) %>%
    count(POLICY_ERA, START_DATE, END_DATE, STATE = CAN_HOSPITAL_STATE, name = "TX") %>%
    mutate(PERIOD_TYPE = "1YR19vs1YR21", .before = everything()) %>%
    arrange(STATE, START_DATE)

txByStatePeriod.v1 %>% head()

PERIOD_TYPE,POLICY_ERA,START_DATE,END_DATE,STATE,TX
<chr>,<chr>,<date>,<date>,<chr>,<int>
1YR19vs1YR21,PRE,2019-01-01,2019-12-31,AL,128
1YR19vs1YR21,POST,2021-01-01,2021-12-31,AL,72
1YR19vs1YR21,PRE,2019-01-01,2019-12-31,AR,43
1YR19vs1YR21,POST,2021-01-01,2021-12-31,AR,45
1YR19vs1YR21,PRE,2019-01-01,2019-12-31,AZ,326
1YR19vs1YR21,POST,2021-01-01,2021-12-31,AZ,311


In [6]:
# count transplants by period/state for '18 + '19 vs. '21 + '22 (using OPTN totals)
txByStatePeriod.v2 <- optn %>%
    filter(CENTER_STATE != "All Center States",
           YEAR %in% 2018:2022, YEAR != 2020,
           REC_AGE %in% c("18-34 Years", "35-49 Years", "50-64 Years", "65 +", "Unknown")) %>%
    left_join(state_code_mapping, by = c("CENTER_STATE" = "STATE_NAME")) %>%
    mutate(POLICY_ERA = if_else(YEAR %in% 2018:2019, "PRE", "POST")) %>%
    group_by(POLICY_ERA) %>%
    mutate(START_DATE = as.Date(paste0(min(YEAR), "-01-01")),
           END_DATE = as.Date(paste0(max(YEAR), "-12-31"))) %>%
    ungroup() %>%
    count(POLICY_ERA, START_DATE, END_DATE, STATE = STATE_CODE, wt = TX, name = "TX") %>%
    mutate(PERIOD_TYPE = "2YR1819vs2YR2122", .before = everything()) %>%
    filter(TX > 0) %>%
    arrange(STATE, START_DATE)

txByStatePeriod.v2 %>% head()

PERIOD_TYPE,POLICY_ERA,START_DATE,END_DATE,STATE,TX
<chr>,<chr>,<date>,<date>,<chr>,<dbl>
2YR1819vs2YR2122,PRE,2018-01-01,2019-12-31,AL,247
2YR1819vs2YR2122,POST,2021-01-01,2022-12-31,AL,155
2YR1819vs2YR2122,PRE,2018-01-01,2019-12-31,AR,80
2YR1819vs2YR2122,POST,2021-01-01,2022-12-31,AR,96
2YR1819vs2YR2122,PRE,2018-01-01,2019-12-31,AZ,607
2YR1819vs2YR2122,POST,2021-01-01,2022-12-31,AZ,646


In [7]:
# defining pre-period (02/03/2018 - 02/03/2020) and post-period (02/04/2020 - 02/03/2022)
# these periods are actually 731 days, but will refer to them as 730-days
unos_periods <- list(pre  = as.Date(c("2018-02-03", "2020-02-03")),
                     post = as.Date(c("2020-02-04", "2022-02-03")))

# count transplants by period/sate for 730-day periods around policy shift (using transplant)
txByStatePeriod.v3 <- transplant %>%
    filter(between(REC_TX_DT_DATE, unos_periods$pre[1], unos_periods$post[2]),
           !is.na(CAN_HOSPITAL_STATE)) %>%
    mutate(POLICY_ERA = case_when(between(REC_TX_DT_DATE, unos_periods$pre[1], unos_periods$pre[2]) ~ "PRE",
                                  between(REC_TX_DT_DATE, unos_periods$post[1], unos_periods$post[2]) ~ "POST"),
           START_DATE = if_else(POLICY_ERA == "PRE", unos_periods$pre[1], unos_periods$post[1]),
           END_DATE = if_else(POLICY_ERA == "PRE", unos_periods$pre[2], unos_periods$post[2])) %>%
    count(POLICY_ERA, START_DATE, END_DATE, STATE = CAN_HOSPITAL_STATE, name = "TX") %>%
    mutate(PERIOD_TYPE = "730Dvs730D", .before = everything()) %>%
    arrange(STATE, START_DATE)

txByStatePeriod.v3 %>% head()

PERIOD_TYPE,POLICY_ERA,START_DATE,END_DATE,STATE,TX
<chr>,<chr>,<date>,<date>,<chr>,<int>
730Dvs730D,PRE,2018-02-03,2020-02-03,AL,245
730Dvs730D,POST,2020-02-04,2022-02-03,AL,162
730Dvs730D,PRE,2018-02-03,2020-02-03,AR,81
730Dvs730D,POST,2020-02-04,2022-02-03,AR,77
730Dvs730D,PRE,2018-02-03,2020-02-03,AZ,613
730Dvs730D,POST,2020-02-04,2022-02-03,AZ,601


### Combine all the different transplant counts

Scaling all periods to be reflective of 1-year average transplant counts (so scaling down v2 + v3)

In [8]:
txChangeComp <- bind_rows(txByStatePeriod.v1, txByStatePeriod.v2, txByStatePeriod.v3) %>%
    mutate(TX_AVG = TX / as.numeric(END_DATE - START_DATE + 1) * 365) %>%
    pivot_wider(id_cols = c("STATE", "PERIOD_TYPE"), names_from = POLICY_ERA, values_from = TX_AVG) %>%
    mutate(TX_N_CHG = POST - PRE,
           TX_PCT_CHG = POST / PRE - 1) %>%
    arrange(STATE, PERIOD_TYPE)

txChangeComp %>% head()

STATE,PERIOD_TYPE,PRE,POST,TX_N_CHG,TX_PCT_CHG
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
AL,1YR19vs1YR21,128.0000,72.00000,-56.000000,-0.43750000
AL,2YR1819vs2YR2122,123.5000,77.50000,-46.000000,-0.37246964
AL,730Dvs730D,122.3324,80.88919,-41.443228,-0.33877551
AR,1YR19vs1YR21,43.0000,45.00000,2.000000,0.04651163
AR,2YR1819vs2YR2122,40.0000,48.00000,8.000000,0.20000000
AR,730Dvs730D,40.4446,38.44733,-1.997264,-0.04938272


In [9]:
txChangeComp.wide <- txChangeComp %>%
    pivot_wider(id_cols = STATE, names_from = PERIOD_TYPE, values_from = matches("TX_")) %>%
    arrange(TX_N_CHG_1YR19vs1YR21)

write.csv(txChangeComp.wide, paste0("output/tables/table7-tx_change_comparison_across_periods.csv"), row.names = F)

txChangeComp.wide %>% head()

STATE,TX_N_CHG_1YR19vs1YR21,TX_N_CHG_2YR1819vs2YR2122,TX_N_CHG_730Dvs730D,TX_PCT_CHG_1YR19vs1YR21,TX_PCT_CHG_2YR1819vs2YR2122,TX_PCT_CHG_730Dvs730D
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PA,-80,-50.0,-51.92886,-0.13675214,-0.08928571,-0.09138840
LA,-65,-78.5,-64.41176,-0.27196653,-0.32304527,-0.26380368
AL,-56,-46.0,-41.44323,-0.43750000,-0.37246964,-0.33877551
TX,-39,33.0,21.47059,-0.04761905,0.04238921,0.02738854
KS,-30,-27.5,-26.96306,-0.37500000,-0.34591195,-0.36241611
NC,-25,-4.5,-18.97401,-0.10204082,-0.02031603,-0.08407080
